In [ ]:
# import needed packages

#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
import numpy as np
import xarray as xr 
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
from matplotlib import colormaps
import cartopy.crs as ccrs #for plotting on map
import cartopy as cart
from datetime import datetime, timedelta
from importlib import reload
import hexbin_functions as hexfunc
import h3
from shapely.geometry import shape, Point

hexfunc = reload(hexfunc)
# test version, it should be version 4
print(h3.__version__)

from analysis_functions_xr import trajectory_length, Haversine_list, calc_tidal_av
from analysis_functions import make_PDF, Haversine
from particle_characteristics_functions import factor_drag_white1991, factor_drag_Schiller1933, factor_drag_morrison2013, Re_particle

# plotstyle: 
plt.style.use('../python_style_Meike.mplstyle')
# markers
color_array = np.array(['c','orange','purple','green','navy'])
marker_array = np.array(['s','o','>','p'])
line_array = np.array(['-','--','-.',':'])
markerline_array = np.array(['-s','--o','-.>',':p'])

Magma = colormaps['magma']
Magmalist = Magma(np.linspace(0.2, 0.8, 3))#Magma(np.linspace(0.2, 0.9, 5))

In [ ]:
# set needed constants
# set needed constants
Rearth = 6371 * 10**3 # in m,
deg2rad = np.pi / 180.
sec_in_hours= 3600
diameter = 0.2 #m
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water

In [ ]:
# filenames
# import simulation data
Replist = np.array([100, 450, 1000])
coriolislist = [True, False]
gradientlist = [True, False]
B = 0.68
tau = 2994.76 #2759.97
runtime =  timedelta(days=30)# timedelta(days=10)
land_handling = 'anti_beaching'
nparticles = 88347 # 52511
chunck_time = 100
loc = 'NWES'
displacement = 300 #m

starttimes = np.array([datetime(2023, 9, 1, 0, 0, 0, 0), #]) #,
                        datetime(2023, 10, 1, 0, 0, 0, 0), #)
                       datetime(2023, 11, 1, 0, 0, 0, 0) ,
                       datetime(2023, 12, 1, 0, 0, 0, 0) ,
                       datetime(2024, 1, 1, 0, 0, 0, 0),
                       datetime(2024, 2, 1, 0, 0, 0, 0)])

date_new = datetime(2023, 9, 1, 0, 0, 0, 0)

base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/'

basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')



basefile_tracer  = (base_directory + '{particle_type}/{loc}_'
                        'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                        'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_{land_handling}.zarr')

basefile_tracer_random = (base_directory + 'tracer_random/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                   '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_d{d:04d}_{land_handling}.zarr')

basefile_MR_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')


particle_types = ['tracer','tracer_random','inertial_SM_drag_Rep','inertial_SM_Rep_constant'] # 
simtype = {'tracer':'tracer','tracer_random':'displaced tracer','inertial_SM_Rep_constant':'SM MR','inertial_Rep_constant':'full MR', 'inertial_SM_drag_Rep': 'flexible Re$_p$'}


basefiles = {'tracer':basefile_tracer,
             'tracer_random':basefile_tracer_random,
             'inertial_SM_drag_Rep':basefile_MR_Rep_drag,
             'inertial_Rep_constant':basefile_Rep_constant,
             'inertial_SM_Rep_constant':basefile_Rep_constant
             }

for Rep in Replist:
    print(f'c({Rep}) = {factor_drag_white1991(Rep):.02f}')


In [ ]:
# read in data
data ={}
for pt in particle_types:
    data[pt]={}
    for coriolis in coriolislist:
        data[pt][coriolis]={}
        for gradient in gradientlist:
            data[pt][coriolis][gradient]={}
            if(pt in  ['tracer','tracer_random','inertial_SM_drag_Rep']):
                data[pt][coriolis][gradient][None]={}
            else:
                for Rep in Replist:
                    data[pt][coriolis][gradient][Rep]={}


for pt in particle_types:
    for coriolis in coriolislist:
        for gradient in gradientlist:
            for starttime in starttimes:
                if(gradient == False and coriolis == False):
                    continue
                # print(starttime)
                endtime = starttime + runtime 
                date = f'{starttime.year:04d}/{starttime.month:02d}'

                if(pt in ('tracer','tracer_random')):
                    if(gradient == True and coriolis == True):
                        file = basefiles[pt].format(loc=loc,
                                                    y_s=starttime.year,
                                                    m_s=starttime.month,
                                                    d_s=starttime.day,
                                                    y_e=endtime.year,
                                                    m_e=endtime.month,
                                                    d_e=endtime.day,
                                                    d=displacement,
                                                    land_handling = land_handling, 
                                                    coriolis = coriolis,
                                                    particle_type = pt)
                        
                        ds = xr.open_dataset(file,
                                            engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['z'],
                                            decode_times=False) #,decode_cf=False)

                        data[pt][coriolis][gradient][None][date]= ds 
                            
                    
                elif(pt == 'inertial_SM_drag_Rep'):
                    # print (pt)
                    file = basefiles[pt].format(loc=loc,
                                                    y_s=starttime.year,
                                                    m_s=starttime.month,
                                                    d_s=starttime.day,
                                                    y_e=endtime.year,
                                                    m_e=endtime.month,
                                                    gradient=gradient,
                                                    d_e=endtime.day,
                                                    B = int(B * 1000), 
                                                    tau = int(tau ),
                                                    land_handling = land_handling, 
                                                    coriolis = coriolis,
                                                    particle_type = pt)
                    ds = xr.open_dataset(file,
                        engine='zarr',
                        chunks={'trajectory':nparticles, 'obs':chunck_time},
                        drop_variables=['z'],
                        decode_times=False) #,decode_cf=False)

                    data[pt][coriolis][gradient][None][date]= ds 
                else:
                    for Rep in Replist:
                        file = basefiles[pt].format(loc=loc,
                                                    y_s=starttime.year,
                                                    m_s=starttime.month,
                                                    d_s=starttime.day,
                                                    y_e=endtime.year,
                                                    m_e=endtime.month,
                                                    d_e=endtime.day,
                                                    gradient=gradient,
                                                    B = int(B * 1000), 
                                                    tau = int(tau ),
                                                    land_handling = land_handling, 
                                                    coriolis = coriolis,
                                                    particle_type = pt,
                                                    Rep = Rep)
                        ds = xr.open_dataset(file,
                                            engine='zarr',
                                            chunks={'trajectory':nparticles, 'obs':chunck_time},
                                            drop_variables=['B','tau','z'],
                                            decode_times=False) #,decode_cf=False)

                        data[pt][coriolis][gradient][Rep][date]= ds 
                        print(Rep)



In [ ]:
pt = 'inertial_SM_drag_Rep'
Rep = None
year=2023
month=9
date = f'{year:04d}/{month:02d}'

fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 25 
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')


ax.plot(data['tracer'][True][True][None][date].lon[idstart,0::1].T,
                data['tracer'][True][True][None][date].lat[idstart,0::1].T,
                '-',
                color='k',zorder=20);

ax.plot(data['tracer_random'][True][True][None][date].lon[idstart,0::1].T,
                data['tracer_random'][True][True][None][date].lat[idstart,0::1].T,
                '-',
                color='grey',zorder=20);

ax.plot(data['inertial_SM_drag_Rep'][False][True][None][date].lon[idstart,0::1].T,
                data['inertial_SM_drag_Rep'][False][True][None][date].lat[idstart,0::1].T,
                '--',
                color=Magmalist[2],zorder=20);

ax.plot(data['inertial_SM_drag_Rep'][True][False][None][date].lon[idstart,0::1].T,
                data['inertial_SM_drag_Rep'][True][False][None][date].lat[idstart,0::1].T,
                '--',
                color='cornflowerblue',zorder=20);

# ax.legend()
ax.legend(['tracer','displaced tracer','gradients forcing','coriolis forcing','gradients and coriolis'],fontsize=18, handlelength = 1.5)#,loc=(1,0.3))

ax.plot(data['inertial_SM_drag_Rep'][True][False][None][date].lon[idstart,0].T,
                data['inertial_SM_drag_Rep'][True][False][None][date].lat[idstart,0].T,
                '>',
                color='darkgreen',zorder=20,markersize=15);

ax.set_xlim(-14.6,-11.2)
ax.set_ylim(54.4,56)


gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
        # ax.set_xlim(-14.6,-10.5)
        # ax.set_ylim(54.4,56.1)


In [ ]:
pt = 'inertial_SM_Rep_constant'
Rep = 100
year=2023
month=9
date = f'{year:04d}/{month:02d}'

fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 25 
ax.add_feature(cart.feature.LAND, facecolor='lightgrey')


ax.plot(data['tracer'][True][True][None][date].lon[idstart,0::1].T,
                data['tracer'][True][True][None][date].lat[idstart,0::1].T,
                '-',
                color='k',zorder=20);

ax.plot(data[pt][False][True][Rep][date].lon[idstart,0::1].T,
                data[pt][False][True][Rep][date].lat[idstart,0::1].T,
                '--',
                color=Magmalist[2],zorder=20);

ax.plot(data[pt][True][False][Rep][date].lon[idstart,0::1].T,
                data[pt][True][False][Rep][date].lat[idstart,0::1].T,
                '--',
                color='cornflowerblue',zorder=20);

# ax.plot(data[pt][True][True][Rep][date].lon[idstart,0::1].T,
#                 data[pt][True][True][Rep][date].lat[idstart,0::1].T,
#                 '--',
#                 color=Magmalist[0],zorder=20);

# ax.legend()
ax.legend(['tracer','gradients forcing','Coriolis forcing','gradients and coriolis'],fontsize=18, handlelength = 1.5,loc=(1,0.3))
ax.set_xlim(-14.6,-11.2)
ax.set_ylim(54.4,55.75)


gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
        linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 18}
gl.ylabel_style =  {'size': 18}
        # ax.set_xlim(-14.6,-10.5)
        # ax.set_ylim(54.4,56.1)


In [ ]:
Blues = colormaps['Blues']



# Take colors at regular intervals spanning the colormap.
bluelist = Blues(np.linspace(0.3, 0.7, 2))



fig,ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
ax.coastlines()
idstart = 6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
idstep = 1
idend = idstart+1 * idstep

ax.add_feature(cart.feature.LAND, facecolor='lightgrey')


# plot selection of trajectories
Blues = colormaps['Blues']
pt = particle_types[1]
Rep = 0
for coriolist, color in zip(coriolislist,bluelist):
        year=2023
        month=9
        date = f'{year:04d}/{month:02d}'
        legend= []
        coriolis = True
        
        ax.coastlines()
        idstart = 25#100# 25# 6008#25#6008 # np.random.randint(0,nparticles)#22#3748 #568#25680#47584#98
        idstep = 1
        idend = idstart+1 * idstep

        ax.add_feature(cart.feature.LAND, facecolor='lightgrey')

        # plot tracer particle
        ax.plot(data['tracer'][coriolis][None][date].lon[idstart,0::1].T,
                data['tracer'][coriolis][None][date].lat[idstart,0::1].T,
                '-',
                color='k',zorder=-20);
        legend.append('tracer')

        # ax.legend(particle_types,loc=(1,0.2))
        pt = 'inertial_SM_Rep_constant'
        Rep = 100
        for coriolis, color in zip(coriolislist,bluelist):
                ax.plot(data[pt][coriolis][Rep][date].lon[idstart:idend:idstep,0::1].T,
                        data[pt][coriolis][Rep][date].lat[idstart:idend:idstep,0::1].T,
                        '--',
                        color=color);

                
        ax.legend(['tracer','coriolis on', 'coriolis off'],loc = 2,fontsize=18, handlelength = 1.5) #, loc=(1,0.2)
        gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0, color='gray', alpha=0.5, linestyle='-')
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 18}
        gl.ylabel_style =  {'size': 18}
        ax.set_xlim(-14.6,-10.5)
        ax.set_ylim(54.4,56.1)


# fig.tight_layout()
fig.savefig('../figures/poster/trajectory_ID25_coriolis.pdf')

In [ ]:
# add tidal average lons and lats 
window=25
startnan=np.full((nparticles,25),np.nan)
endnan=np.full((nparticles,80),np.nan)
Tend=720

for starttime in starttimes:
    year = starttime.year
    month = starttime.month
    date = f'{year:04d}/{month:02d}'
    for coriolis in coriolislist:
        for gradient in gradientlist: 
            if(gradient == False and coriolis == False):
                continue
            for pt in particle_types:
                # print(pt)
                
                if(pt == 'inertial_SM_Rep_constant'):
                    for Rep in Replist:
                        da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][gradient][Rep][date].lon,window)
                        da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][gradient][Rep][date].lat,window)

                        data[pt][coriolis][gradient][Rep][date] = data[pt][coriolis][gradient][Rep][date].assign(lon_tidal_av = da_lon_tidal_av)
                        data[pt][coriolis][gradient][Rep][date] = data[pt][coriolis][gradient][Rep][date].assign(lat_tidal_av = da_lat_tidal_av)
                elif(pt in ('tracer','tracer_random')):
                    if(gradient == True and coriolis == True):
                        da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][gradient][None][date].lon,window)
                        da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][gradient][None][date].lat,window)

                        data[pt][coriolis][gradient][None][date] = data[pt][coriolis][gradient][None][date].assign(lon_tidal_av = da_lon_tidal_av)
                        data[pt][coriolis][gradient][None][date] = data[pt][coriolis][gradient][None][date].assign(lat_tidal_av = da_lat_tidal_av)
                
                else:
                    da_lon_tidal_av = calc_tidal_av(data[pt][coriolis][gradient][None][date].lon,window)
                    da_lat_tidal_av = calc_tidal_av(data[pt][coriolis][gradient][None][date].lat,window)

                    data[pt][coriolis][gradient][None][date] = data[pt][coriolis][gradient][None][date].assign(lon_tidal_av = da_lon_tidal_av)
                    data[pt][coriolis][gradient][None][date] = data[pt][coriolis][gradient][None][date].assign(lat_tidal_av = da_lat_tidal_av)
                

In [ ]:
# along track distance
for gradient in gradientlist:
    for coriolis in coriolislist:
        if(coriolis == False and gradient == False ):
            continue
        for pt in particle_types: # loop over all particle types
            for starttime in starttimes: # loop over all starttime
                year = starttime.year
                month = starttime.month
                date = f'{year:04d}/{month:02d}'
                if(pt == 'inertial_SM_Rep_constant'):
                    for Rep in Replist:
                        ds =data[pt][coriolis][gradient][Rep][date]
                        traj_length = trajectory_length(ds.lon, ds.lat)
                        ds = ds.assign(traj_length = traj_length)
                        traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:], ds.lat_tidal_av[:,window:])
                        data[pt][coriolis][gradient][Rep][date] = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
                elif(pt in ('tracer','tracer_random')):
                    if(coriolis == True and gradient == True):
                        ds=data[pt][coriolis][gradient][None][date]
                        traj_length = trajectory_length(ds.lon, ds.lat)
                        ds = ds.assign(traj_length = traj_length)
                        traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:],ds.lat_tidal_av[:,window:])
                        data[pt][coriolis][gradient][None][date] = ds.assign(traj_length_tidal_av = traj_length_tidal_av)

            
                else:
                    ds=data[pt][coriolis][gradient][None][date]
                    traj_length = trajectory_length(ds.lon, ds.lat)
                    ds = ds.assign(traj_length = traj_length)
                    traj_length_tidal_av = trajectory_length(ds.lon_tidal_av[:,window:],ds.lat_tidal_av[:,window:])
                    data[pt][coriolis][gradient][None][date] = ds.assign(traj_length_tidal_av = traj_length_tidal_av)
                    


In [ ]:
# think about how to plot difference 
# plot PDF difference trajecory length and rest of particles
i=0
colorlist=['k','grey','cornflowerblue','blue']
markers = ['-','-','--','-.']
coriolis = True
gradient = False
Tmax=719
fig, ax = plt.subplots()
data_arrays_tracer = list(data['tracer'][True][True][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
legend = []
for pt, color, marker in zip(particle_types[1:],colorlist[1:],markers[1:]):





    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][gradient][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            
            diff = combined_data_Rep.traj_length_tidal_av - combined_data_tracer.traj_length_tidal_av
            average_data = diff.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
            ax.plot(bins,pdf,'-.',color=color2, zorder = i)
            legend.append('Re$_p$ ='+' {:d}'.format(Rep))
            i-=5

    else:
        if(pt == 'tracer_random'):
            data_arrays = list(data[pt][True][True][None].values())
        else:
            data_arrays = list(data[pt][coriolis][gradient][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        diff = combined_data.traj_length_tidal_av - combined_data_tracer.traj_length_tidal_av
        average_data = diff.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
        ax.plot(bins,pdf,marker,color=color)
        legend.append(simtype[pt])

    # ax.set_title(simtype[pt])
    ax.legend(legend,handlelength = 1.5)#,loc=(1,0.2))
    ax.set_ylabel("PDF")
    ax.set_xlabel("tidal av trajecory length difference to tracer [km]")
    ax.set_ylim(-0.005,0.255)
    ax.set_title('coriolis forcing',fontsize=18)
    # ax.set_yscale('log')

In [ ]:
# think about how to plot difference 
# plot PDF difference trajecory length and rest of particles
i=0
colorlist=['k','grey','cornflowerblue','blue']
markers = ['-','-','--','-.']
coriolis = False
gradient = True
Tmax=719
fig, ax = plt.subplots()
data_arrays_tracer = list(data['tracer'][True][True][None].values())
combined_data_tracer = xr.concat(data_arrays_tracer, dim="starting_days")
legend = []
for pt, color, marker in zip(particle_types[1:],colorlist[1:],markers[1:]):





    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][gradient][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            
            diff = combined_data_Rep.traj_length_tidal_av - combined_data_tracer.traj_length_tidal_av
            average_data = diff.mean(dim="starting_days",skipna=True)
            bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
            ax.plot(bins,pdf,'-.',color=color2, zorder = i)
            legend.append('Re$_p$ ='+' {:d}'.format(Rep))
            i-=5
    else:
        if(pt == 'tracer_random'):
            data_arrays = list(data[pt][True][True][None].values())
        else:
            data_arrays = list(data[pt][coriolis][gradient][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        diff = combined_data.traj_length_tidal_av - combined_data_tracer.traj_length_tidal_av
        average_data = diff.mean(dim="starting_days",skipna=True)
        bins, pdf = make_PDF( average_data[:,Tmax-1].values.flatten(),51, norm=True,vmin = -20,vmax = 20)
        ax.plot(bins,pdf,marker,color=color)
        legend.append(simtype[pt])

    # ax.set_title(simtype[pt])
    ax.legend(legend,handlelength = 1.5)#,loc=(1,0.2))
    ax.set_ylabel("PDF")
    ax.set_xlabel("tidal av trajecory length difference to tracer [km]")
    # ax.set_ylim(-0.005,0.255)
    # ax.set_yscale('log')
    ax.set_title('gradient forcing',fontsize=18)

In [ ]:
# Relative dispersion

for coriolis in coriolislist:
    for gradient in gradientlist:
        if(coriolis == False and gradient == False):
            continue
        for starttime in starttimes:
            print(starttime)
            year = starttime.year
            month = starttime.month
            date = f'{year:04d}/{month:02d}' #'inertial_SM_drag_Rep','inertial_Rep_constant','inertial_SM_Rep_constant']
            
            ds_tracer = data['tracer'][True][True][None][date]
            for pt in particle_types[1:]:
                if(pt == 'inertial_SM_Rep_constant'):
                    for Rep in Replist:
                        ds_Rep = data[pt][coriolis][gradient][Rep][date]
                        dist= Haversine(ds_Rep['lon'],ds_Rep['lat'],
                                            ds_tracer['lon'],ds_tracer['lat'])
                        data[pt][coriolis][gradient][Rep][date]=data[pt][coriolis][gradient][Rep][date].assign(dist = dist)
                        data[pt][coriolis][gradient][Rep][date]=data[pt][coriolis][gradient][Rep][date].rename({'dist':'dist_to_tracer'})

                        #tial av
                        dist_tidal_av= Haversine(ds_Rep['lon_tidal_av'],ds_Rep['lat_tidal_av'],
                                            ds_tracer['lon_tidal_av'],ds_tracer['lat_tidal_av'])
                        data[pt][coriolis][gradient][Rep][date]=data[pt][coriolis][gradient][Rep][date].assign(dist_tidal_av = dist_tidal_av)
                        data[pt][coriolis][gradient][Rep][date]=data[pt][coriolis][gradient][Rep][date].rename({'dist_tidal_av':'dist_to_tracer_tidal_av'})
            
                elif(pt == 'tracer_random'):
                    if(gradient == True and coriolis == True):
                        ds = data[pt][coriolis][gradient][None][date]
                        dist= Haversine(ds['lon'],ds['lat'],
                                                ds_tracer['lon'],ds_tracer['lat'])
                        data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].assign(dist = dist)
                        data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].rename({'dist':'dist_to_tracer'})

                        #tial av
                        dist_tidal_av= Haversine(ds['lon_tidal_av'],ds['lat_tidal_av'],
                                                ds_tracer['lon_tidal_av'],ds_tracer['lat_tidal_av'])
                        data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].assign(dist_tidal_av = dist_tidal_av)
                        data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].rename({'dist_tidal_av':'dist_to_tracer_tidal_av'})
                else:
                    ds = data[pt][coriolis][gradient][None][date]
                    dist= Haversine(ds['lon'],ds['lat'],
                                            ds_tracer['lon'],ds_tracer['lat'])
                    data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].assign(dist = dist)
                    data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].rename({'dist':'dist_to_tracer'})

                    #tial av
                    dist_tidal_av= Haversine(ds['lon_tidal_av'],ds['lat_tidal_av'],
                                            ds_tracer['lon_tidal_av'],ds_tracer['lat_tidal_av'])
                    data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].assign(dist_tidal_av = dist_tidal_av)
                    data[pt][coriolis][gradient][None][date]=data[pt][coriolis][gradient][None][date].rename({'dist_tidal_av':'dist_to_tracer_tidal_av'})



In [ ]:
# plot distribution final distance between particles
# slow manifold finds bigger seperation distance (as expected because more oscillations)
tfinal = 799
gradient = False
coriolis = True
markerlist=np.array(['-','--','-.',':','-','--','-.',':'])

i=0
Tmax=720
# cmap = plt.colormaps['magma']
# magma_colors = cmap(np.linspace(0.2, 0.9,4))
fig,ax=plt.subplots()
legend=[]
for pt,color,marker in zip(particle_types[1:],colorlist[1:],markerlist):   

    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][gradient][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            mean=combined_data_Rep['dist_to_tracer'].mean(dim='trajectory',skipna=True)
            # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
            average_mean = mean.mean(dim="starting_days",skipna=True)
            average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
            
            tlist=np.arange(0,Tmax,1)/24
        
            # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
            ax.plot(tlist[:720], average_mean[:720],'-.', color=color2)
            ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color2,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
        
            legend.append('Re$_p=$'+ ' {:d}'.format(Rep))
    else:
        if(pt == 'tracer_random'):
            data_arrays = list(data[pt][True][True][None].values())
        else:
            data_arrays = list(data[pt][coriolis][gradient][None].values())

        combined_data = xr.concat(data_arrays, dim="starting_days") 
        mean=combined_data['dist_to_tracer'].mean(dim='trajectory',skipna=True)
        # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
        average_mean = mean.mean(dim="starting_days",skipna=True)
        average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
        
        tlist=np.arange(0,Tmax,1)/24
    
        # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
        ax.plot(tlist[:720], average_mean[:720],marker, color=color,zorder=20)
        ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    
        legend.append(simtype[pt])

    ax.legend(legend,fontsize=18,handlelength=1.5)
    ax.set_xlabel('t [days]')
    ax.set_ylabel('distance to tracer [km]')
    # ax.set_yscale('log')
    # ax.set_xscale('log')
    ax.set_title('coriolis forcing',fontsize=18)

In [ ]:
# plot distribution final distance between particles
# slow manifold finds bigger seperation distance (as expected because more oscillations)
tfinal = 799
gradient = True
coriolis = False
markerlist=np.array(['-','--','-.',':','-','--','-.',':'])

i=0
Tmax=720
# cmap = plt.colormaps['magma']
# magma_colors = cmap(np.linspace(0.2, 0.9,4))
fig,ax=plt.subplots()
legend=[]
for pt,color,marker in zip(particle_types[1:],colorlist[1:],markerlist):   

    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][gradient][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            mean=combined_data_Rep['dist_to_tracer'].mean(dim='trajectory',skipna=True)
            # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
            average_mean = mean.mean(dim="starting_days",skipna=True)
            average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
            
            tlist=np.arange(0,Tmax,1)/24
        
            # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
            ax.plot(tlist[:720], average_mean[:720],'-.', color=color2)
            ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color2,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
        
            legend.append('Re$_p=$'+ ' {:d}'.format(Rep))
    else:
        if(pt == 'tracer_random'):
            data_arrays = list(data[pt][True][True][None].values())
        else:
            data_arrays = list(data[pt][coriolis][gradient][None].values())

        combined_data = xr.concat(data_arrays, dim="starting_days") 
        mean=combined_data['dist_to_tracer'].mean(dim='trajectory',skipna=True)
        # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
        average_mean = mean.mean(dim="starting_days",skipna=True)
        average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
        
        tlist=np.arange(0,Tmax,1)/24
    
        # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
        ax.plot(tlist[:720], average_mean[:720],marker, color=color,zorder=20)
        ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    
        legend.append(simtype[pt])

    ax.legend(legend,fontsize=18,handlelength=1.5)
    ax.set_xlabel('t [days]')
    ax.set_ylabel('distance to tracer [km]')
    # ax.set_yscale('log')
    # ax.set_xscale('log')
    ax.set_title('gradient forcing',fontsize=18)

In [ ]:
# plot distribution final distance between particles
# slow manifold finds bigger seperation distance (as expected because more oscillations)
tfinal = 799
gradient = True
coriolis = False
markerlist=np.array(['-','--','-.',':','-','--','-.',':'])

i=0
Tmax=720
# cmap = plt.colormaps['magma']
# magma_colors = cmap(np.linspace(0.2, 0.9,4))
fig,ax=plt.subplots()
legend=[]
for pt,color,marker in zip(particle_types[1:],colorlist[2:],markerlist[1:]):   

    if(pt == 'inertial_SM_Rep_constant'):
        for Rep, color2 in zip(Replist,Magmalist):
            data_arrays_Rep = list(data[pt][coriolis][gradient][Rep].values())
            combined_data_Rep = xr.concat(data_arrays_Rep, dim="starting_days") 
            mean=combined_data_Rep['dist_to_tracer'].mean(dim='trajectory',skipna=True)
            # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
            average_mean = mean.mean(dim="starting_days",skipna=True)
            average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
            
            tlist=np.arange(0,Tmax,1)/24
        
            # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
            ax.plot(tlist[:720], average_mean[:720],'-.', color=color2)
            ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color2,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
        
            legend.append('Re$_p=$'+ ' {:d}'.format(Rep))
    else:
        data_arrays = list(data[pt][coriolis][gradient][None].values())
        combined_data = xr.concat(data_arrays, dim="starting_days") 
        mean=combined_data['dist_to_tracer'].mean(dim='trajectory',skipna=True)
        # std =combined_data_Rep['dist_to_tracer'].std(dim='trajectory',skipna=True)
        average_mean = mean.mean(dim="starting_days",skipna=True)
        average_std = mean.std(dim="starting_days",skipna=True)/np.sqrt(starttimes.size)
        
        tlist=np.arange(0,Tmax,1)/24
    
        # ax.errorbar(tlist[:720], average_mean[:720],average_std[:720], fmt='-', color=color)
        ax.plot(tlist[:720], average_mean[:720],marker, color=color,zorder=20)
        ax.fill_between(tlist[:720], average_mean[:720]+average_std[:720],  average_mean[:720]-average_std[:720],color=color,alpha=0.2,label='_nolegend_')#,alhpa=0.2)
    
        legend.append(simtype[pt])

    ax.legend(legend,fontsize=18,handlelength=1.5)
    ax.set_xlabel('t [days]')
    ax.set_ylabel('distance to tracer [km]')
    # ax.set_yscale('log')
    # ax.set_xscale('log')

In [ ]:
# show final shapshot
startime = starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'
pt = 'inertial_SM_Rep_constant'
Tmax=700
fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})

ax.scatter(data['tracer'][coriolis][None][date].lon[:,Tmax],data['tracer'][coriolis][None][date].lat[:,Tmax],marker='o',s=2,alpha=0.1,color='k')
ax.coastlines()
ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
            linewidth=0, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 20}
gl.ylabel_style =  {'size': 20}

Rep = 0
for coriolis, color in zip(coriolislist,color_array):
    fig, ax = plt.subplots(subplot_kw={'projection':ccrs.PlateCarree()})
    ax.scatter(data[pt][coriolis][Rep][date].lon[:,Tmax],data[pt][coriolis][Rep][date].lat[:,Tmax],marker='o',s=2,alpha=0.1,color=color)
    ax.coastlines()
    ax.add_feature(cart.feature.LAND,facecolor='lightgrey')
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                linewidth=0, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xlabel_style = {'size': 20}
    gl.ylabel_style =  {'size': 20}




In [ ]:
# make histogram

hex_res = 5
sim_domain = { 
    "type":"Polygon",
    "coordinates": [
   [[9.977004051208496-0.5,61.28188705444336-0.5],
     [-15.996014595031738+0.5,61.28188705444336-0.5],
     [-15.996014595031738+0.5,46.00364303588867+0.5],
     [9.977004051208496-0.5,46.00364303588867+0.5],
     [9.977004051208496-0.5,61.28188705444336-0.5]]
     ]}
hexagons = h3.polygon_to_cells(h3.geo_to_h3shape(sim_domain), hex_res)
hexbin_grid = hexfunc.hexGrid(hexagons, h3_res=hex_res)
hexbin_grid.process_hexagons()

probability_set = {}
Tmax=719

for pt in particle_types:
    probability_set[pt]={}
    for coriolis in coriolislist:
        probability_set[pt][coriolis]={}
        if(pt in  ['tracer','inertial_SM_drag_Rep']):
            probability_set[pt][coriolis][None]={}
        else:
            for Rep in Replist:
                probability_set[pt][coriolis][Rep]={}

for pt in particle_types:
    for coriolis in coriolislist:
        for starttime in starttimes:
            print(starttime)
            endtime = starttime + runtime 
            date = f'{starttime.year:04d}/{starttime.month:02d}'
            if(pt == 'tracer'):
                ds =data[pt][True][None][date]
                probability_set[pt][True][None][date] =(hexbin_grid.count_2d(ds.lon[:, Tmax].values,ds.lat[:,Tmax].values, normalize=False))
            elif(pt == 'inertial_SM_drag_Rep'):
                for coriolis in coriolislist:
                    ds =data[pt][coriolis][None][date]
                    probability_set[pt][coriolis][None][date] =(hexbin_grid.count_2d(ds.lon[:, Tmax].values,ds.lat[:,Tmax].values, normalize=False))
            else:
                 for coriolis in coriolislist:
                     for Rep in Replist:
                        ds =data[pt][coriolis][Rep][date]
                        probability_set[pt][coriolis][Rep][date] =(hexbin_grid.count_2d(ds.lon[:, Tmax].values,ds.lat[:,Tmax].values, normalize=False))


In [ ]:
probability_set_time={}
starttime=starttimes[0]
year = starttime.year
month = starttime.month
date = f'{year:04d}/{month:02d}'

pt = 'tracer'
Tlist = [0,24*15-1,719]
rep = 0
for t in Tlist:
    print(t)
    ds =data[pt][True][None][date]
    probability_set_time[t] = (hexbin_grid.count_2d(ds.lon[:, t].values,ds.lat[:,t].values, normalize=False))

In [ ]:
fig,ax = plt.subplots()
fig2,ax2 = plt.subplots()
norm =7
tlegend=[]
for t in Tlist:
    bins, pdf = make_PDF(probability_set_time[t][probability_set_time[0]>0],nbins=261,norm=True,vmin=-0.5,vmax=260.5)
    ax.plot(bins[1:]/norm,pdf[1:],'--o')
    dx = bins[2]-bins[1]
    ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx))
    tlegend.append(f'T = {(t+1)/24:.02f} days')
ax.axvline(12,color='grey',zorder=-20)
ax.legend(tlegend)
ax.set_xlim(-0.1,5)
ax.set_xlabel('particle density')
ax.set_ylabel('PDF')
ax.axvline(1,color='grey',zorder=-20)
ax2.axvline(1,color='grey',zorder=-20)
ax2.legend(tlegend)
ax2.set_xscale('log')

In [ ]:
fig, ax =plt.subplots()
fig2, ax2 =plt.subplots()
legend=[]
for pt in particle_types:

    
    norm = 7
    nbins = 81
    max = 81.5
    coriolis =True
    markers=['-','--']
    # starttime=starttimes[1]
    if(pt == 'tracer'):
        data_arrays_Rep = list(probability_set['tracer'][coriolis][None].values())
        filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
        print(np.max(filtered_data)/norm)
        mean = np.mean(np.array([np.mean(data) for data in filtered_data]))
        bins_tracer,pdf_tracer = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
        ax.plot(bins_tracer[1:]/norm,pdf_tracer[1:],'-',color='k',zorder=-20)
        dx = bins[2]-bins[1]
        cdf_tracer=np.cumsum(pdf[1:]*dx)
        ax2.plot(bins_tracer[1:]/norm,np.cumsum(pdf_tracer[1:]*dx),'-',color='k')
        legend.append('tracer')

        # bins,pdf = make_PDF(probability_set_tracer,norm=False,bins=100,min=0,max=100)
    elif(pt == 'inertial_SM_drag_Rep'):
        for coriolis, color in zip(coriolislist,bluelist):
            data_arrays_Rep = list(probability_set[pt][coriolis][None].values())
            filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
            print(np.max(filtered_data))
            bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
            ax.plot(bins[1:]/norm,pdf[1:],'--',color=color)
            dx = bins[2]-bins[1]
            ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'--',color=color)
            # legend.append(simtype[pt]+', $Re_p$ = {:d}'.format(Rep))
    else:
        continue
# ax.set_yscale('log')
ax.legend(['tracer','coriolis on', 'coriolis off'])
# ax.set_xlim(-0.1,3)
ax.axvline(1,color='grey',zorder=-20)
ax2.legend(['tracer','coriolis on', 'coriolis off'])
# ax2.set_xscale('log')
# ax2.set_yscale('log')

ax2.axvline(1,color='grey',zorder=-20)
# ax2.set_xlim(10,15)
# ax2.set_ylim(0.6,0.8)
ax.set_xlabel('particle density')
ax.set_ylabel('PDF')
# ax.set_title(simtype[pt])



ax2.set_xlabel('particle density')
ax2.set_ylabel('CDF')
# ax2.set_yscale('log')

In [ ]:
bins_test, pdf_test =make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True)
print(bins_test[0])
print(bins_test[-1])

In [ ]:
fig, ax =plt.subplots()
fig2, ax2 =plt.subplots()
legend=[]
for pt in particle_types:

    
    norm = 7
    nbins = 1101
    max = 1100.5
    coriolis =True
    markers=['-','--']
    # starttime=starttimes[1]
    if(pt == 'tracer'):
        print('hello')
        data_arrays_Rep = list(probability_set['tracer'][coriolis][None].values())
        filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
        mean = np.mean(np.array([np.mean(data) for data in filtered_data]))
        bins_tracer,pdf_tracer = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
        # ax.plot(bins[1:]/norm,pdf[1:],'-',color='k',zorder=-20)
        dx = bins[2]-bins[1]
        cdf_tracer=np.cumsum(pdf[1:]*dx)
        # ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'-',color='k')
        # legend.append('tracer')

        # bins,pdf = make_PDF(probability_set_tracer,norm=False,bins=100,min=0,max=100)
    elif(pt == 'inertial_SM_drag_Rep'):
        
        for coriolis, color in zip(coriolislist,bluelist):
            data_arrays_Rep = list(probability_set[pt][coriolis][None].values())
            filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
            
            bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
            ax.plot(bins[1:]/norm,pdf[1:]-pdf_tracer[1:],'--',color=color)
            dx = bins[2]-bins[1]
            ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx) - cdf_tracer,'--',color=color)
            # legend.append(simtype[pt]+', $Re_p$ = {:d}'.format(Rep))
    else:
        continue
# ax.set_yscale('log')
ax.legend(['coriolis on', 'coriolis off'])
ax.set_xlim(-0.1,3)
ax.axvline(1,color='grey',zorder=-20)
ax2.legend(['coriolis on', 'coriolis off'])
ax2.set_xscale('log')
# ax2.set_yscale('log')

ax2.axvline(1,color='grey',zorder=-20)
# ax2.set_xlim(10,15)
# ax2.set_ylim(0.6,0.8)
ax.set_xlabel('normalized particle density')
ax.set_ylabel('PDF - tracer PDF')
# ax.set_title(simtype[pt])



ax2.set_xlabel('normalized particle density')
ax2.set_ylabel('CDF - tracer CDF')

In [ ]:
fig, ax =plt.subplots()
fig2, ax2 =plt.subplots()
legend=[]
bluelist4 = Blues(np.linspace(0.3, 0.95, 4))
for pt in particle_types:

    
    norm = 7
    nbins = 201
    max = 200.5
    coriolis =True
    markers=['-','--']
    # starttime=starttimes[1]
    if(pt == 'tracer'):
        data_arrays_Rep = list(probability_set['tracer'][coriolis][None].values())
        filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
        mean = np.mean(np.array([np.mean(data) for data in filtered_data]))
        bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
        ax.plot(bins[1:]/norm,pdf[1:],'-',color='k',zorder=-20)
        dx = bins[2]-bins[1]
        ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'-',color='k')
        legend.append('tracer')

        # bins,pdf = make_PDF(probability_set_tracer,norm=False,bins=100,min=0,max=100)
    elif(pt == 'inertial_SM_Rep_constant'):
        coriolis = True
        for Rep, color in zip(Replist,bluelist4):
            data_arrays_Rep = list(probability_set[pt][coriolis][Rep].values())
            filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
            print(np.max(filtered_data))
            bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
            ax.plot(bins[1:]/norm,pdf[1:],'--',color=color)
            dx = bins[2]-bins[1]
            ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'--',color=color)
            legend.append('Re$_p$ = {:d}'.format(Rep))
    else:
        continue
# ax.set_yscale('log')
ax.legend(legend,handlelength = 1.5)
ax.set_xlim(-0.1,3)
# ax.axvline(1,color='grey',zorder=-20)
ax2.legend(legend,handlelength = 1.5)
ax2.set_xscale('log')
# ax2.set_yscale('log')

# ax2.axvline(1,color='grey',zorder=-20)
# ax2.set_xlim(10,15)
# ax2.set_ylim(0.6,0.8)
ax.set_xlabel('particle density')
ax.set_ylabel('PDF')
# ax.set_title(simtype[pt])



ax2.set_xlabel('normalized particle density ')
ax2.set_ylabel('CDF')

In [ ]:
fig, ax =plt.subplots()
fig2, ax2 =plt.subplots()
legend=[]
bluelist4 = Blues(np.linspace(0.3, 0.95, 4))
for pt in particle_types:

    
    norm = 7
    nbins = 201
    max = 200.5
    coriolis =True
    markers=['-','--']
    # starttime=starttimes[1]
    if(pt == 'tracer'):
        data_arrays_Rep = list(probability_set['tracer'][coriolis][None].values())
        filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
        mean = np.mean(np.array([np.mean(data) for data in filtered_data]))
        bins_tracer,pdf_tracer = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
        # ax.plot(bins[1:]/norm,pdf[1:],'-',color='k',zorder=-20)
        dx = bins_tracer[2]-bins_tracer[1]
        cdf_tracer= np.cumsum(pdf[1:]*dx)
        # ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'-',color='k')
        # legend.append('tracer')

        # bins,pdf = make_PDF(probability_set_tracer,norm=False,bins=100,min=0,max=100)
    elif(pt == 'inertial_SM_Rep_constant'):
        coriolis = True
        for Rep, color in zip(Replist,bluelist4):
            data_arrays_Rep = list(probability_set[pt][coriolis][Rep].values())
            filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]

            bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
            ax.plot(bins[1:]/norm,pdf[1:]-pdf_tracer[1:],'--',color=color)
            dx = bins[2]-bins[1]
            ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx)-cdf_tracer,'--',color=color)
            legend.append('Re$_p$ = {:d}'.format(Rep))
    else:
        continue
# ax.set_yscale('log')
ax.legend(legend,handlelength = 1.5)
ax.set_xlim(-0.1,3)
# ax.axvline(1,color='grey',zorder=-20)
ax2.legend(legend,handlelength = 1.5)
ax2.set_xscale('log')
# ax2.set_yscale('log')

# ax2.axvline(1,color='grey',zorder=-20)
# ax2.set_xlim(10,15)
# ax2.set_ylim(0.6,0.8)
ax.set_xlabel('particle density')
ax.set_ylabel('PDF')
# ax.set_title(simtype[pt])



ax2.set_xlabel('normalized particle density ')
ax2.set_ylabel('CDF')

In [ ]:
fig, ax =plt.subplots()
fig2, ax2 =plt.subplots()
legend=[]
bluelist4 = Blues(np.linspace(0.3, 0.95, 4))


    
norm = 7
nbins = 201
max = 200.5
coriolis =True
markers=['-','--']
# starttime=starttimes[1]
pt = 'tracer'
data_arrays_Rep = list(probability_set['tracer'][coriolis][None].values())
filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
mean = np.mean(np.array([np.mean(data) for data in filtered_data]))
bins_tracer,pdf_tracer = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
ax.plot(bins[1:]/norm,pdf[1:],'-',color='k',zorder=-20)
dx =1 # bins_tracer[2]-bins_tracer[1]
cdf_tracer= np.cumsum(pdf[1:]*dx)
ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'-',color='k')
# legend.append('tracer')

# bins,pdf = make_PDF(probability_set_tracer,norm=False,bins=100,min=0,max=100)
pt ='inertial_SM_Rep_constant'
coriolis = True
Rep = 0

data_arrays_Rep = list(probability_set[pt][coriolis][Rep].values())
filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]

bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
ax.plot(bins[1:]/norm,pdf[1:],'--',color=bluelist[0])
dx = 1 #bins[2]-bins[1]
ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'--',color=bluelist[0])
legend.append('Re$_p$ = {:d}'.format(Rep))


pt = 'inertial_SM_drag_Rep'
data_arrays_Rep = list(probability_set[pt][True][None].values())
filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]

bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
ax.plot(bins[1:]/norm,pdf[1:],'--',color=bluelist[1])
dx = 1 #bins[2]-bins[1]
ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'--',color=bluelist[1])
# ax.set_yscale('log')
ax.legend(['tracer','Re$_p$=100','Re$_p$ flexible'],handlelength = 1.5)
ax.set_xlim(-0.1,3)
# ax.axvline(1,color='grey',zorder=-20)
ax2.legend(['tracer','Re$_p$=100','Re$_p$ flexible'],handlelength = 1.5)
ax2.set_xscale('log')
# ax2.set_yscale('log')

# ax2.axvline(1,color='grey',zorder=-20)
# ax2.set_xlim(10,15)
# ax2.set_ylim(0.6,0.8)
ax.set_xlabel('normalized particle density')
ax.set_ylabel('PDF')
# ax.set_title(simtype[pt])



ax2.set_xlabel('normalized particle density ')
ax2.set_ylabel('CDF')

In [ ]:
fig, ax =plt.subplots()
fig2, ax2 =plt.subplots()
legend=[]
bluelist4 = Blues(np.linspace(0.3, 0.95, 4))


    
norm = 7
nbins = 201
max = 200.5
coriolis =True
markers=['-','--']
# starttime=starttimes[1]
pt = 'tracer'
data_arrays_Rep = list(probability_set['tracer'][coriolis][None].values())
filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]
mean = np.mean(np.array([np.mean(data) for data in filtered_data]))
bins_tracer,pdf_tracer = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
# ax.plot(bins[1:]/norm,pdf[1:],'-',color='k',zorder=-20)
dx = bins_tracer[2]-bins_tracer[1]
cdf_tracer= np.cumsum(pdf[1:]*dx)
# ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'-',color='k')
# legend.append('tracer')

# bins,pdf = make_PDF(probability_set_tracer,norm=False,bins=100,min=0,max=100)
pt ='inertial_SM_Rep_constant'
coriolis = True
Rep = 100

data_arrays_Rep = list(probability_set[pt][coriolis][Rep].values())
filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]

bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
ax.plot(bins[1:]/norm,pdf[1:]-pdf_tracer[1:],'--',color=bluelist[0])
dx = bins[2]-bins[1]
ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'--',color=bluelist[0])
legend.append('Re$_p$ = {:d}'.format(Rep))


pt = 'inertial_SM_drag_Rep'
data_arrays_Rep = list(probability_set[pt][True][None].values())
filtered_data = [ data[probability_set_time[0]>0] for data in data_arrays_Rep]

bins,pdf = make_PDF(np.array(filtered_data).flatten(),nbins=nbins,norm=True,vmin=-0.5,vmax=max)
ax.plot(bins[1:]/norm,pdf[1:],'--',color=bluelist[1])
dx = bins[2]-bins[1]
ax2.plot(bins[1:]/norm,np.cumsum(pdf[1:]*dx),'--',color=bluelist[1])
# ax.set_yscale('log')
ax.legend(['Re$_p$=100','Re$_p$ flexible'],handlelength = 1.5)
ax.set_xlim(-0.1,3)
# ax.axvline(1,color='grey',zorder=-20)
ax2.legend(['Re$_p$=100','Re$_p$ flexible'],handlelength = 1.5)
ax2.set_xscale('log')
# ax2.set_yscale('log')

# ax2.axvline(1,color='grey',zorder=-20)
# ax2.set_xlim(10,15)
# ax2.set_ylim(0.6,0.8)
ax.set_xlabel('normalized particle density')
ax.set_ylabel('PDF')
# ax.set_title(simtype[pt])



ax2.set_xlabel('normalized particle density ')
ax2.set_ylabel('CDF')